In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
dataset_path = "/content/drive/MyDrive/Train"
os.listdir(dataset_path)

['Traffic light',
 'Sheep',
 'Laptop',
 'Motorcycle',
 'Orange',
 'Umbrella',
 'Vase',
 'Knife',
 'open_doors',
 'Person',
 'chairs',
 'Elephant',
 'Cow',
 'Horse',
 'Handbag',
 'Cup',
 'Fire hydrant',
 'fruits',
 'closed_doors',
 'Dog',
 'Boat',
 'Cat',
 'Book',
 'Car',
 'Bus',
 'Bird',
 'Carrot',
 'Bear',
 'Bottle',
 'Bicycle',
 'Airplane',
 'Apple',
 'banana']

In [8]:
from PIL import Image
import os

def resize_images(input_folder, output_folder, size=(224, 224)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, dirs, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith(('.png', '.jpg', '.jpeg')):  # تأكد من أنك تتعامل مع ملفات الصور فقط
                img_path = os.path.join(root, filename)
                img = Image.open(img_path)
                img = img.resize(size)

                # تحويل الصورة إلى وضع RGB إذا كانت RGBA، P، أو LA
                if img.mode in ['RGBA', 'P', 'LA']:
                    img = img.convert('RGB')
                elif img.mode not in ['RGB', 'L']:
                    img = img.convert('RGB')  # تحويل أي وضع آخر إلى RGB

                # حفظ الصورة في المجلد الجديد مع الاحتفاظ بالمسار
                relative_path = os.path.relpath(img_path, input_folder)
                output_path = os.path.join(output_folder, relative_path)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                img.save(output_path)

resize_images('/content/drive/MyDrive/Train', '/content/drive/MyDrive/Tra')

In [9]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_data(base_dir, train_dir, test_dir, test_size=0.2):
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    # استعراض جميع الفئات (المجلدات)
    classes = os.listdir(base_dir)

    for class_name in classes:
        class_path = os.path.join(base_dir, class_name)
        if os.path.isdir(class_path):
            # الحصول على قائمة الصور في الفئة
            images = os.listdir(class_path)
            # تقسيم الصور إلى مجموعة تدريب واختبار
            train_images, test_images = train_test_split(images, test_size=test_size, random_state=42)

            # إنشاء المجلدات للفئات في مجموعة التدريب والاختبار
            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

            # نقل الصور إلى المجلدات المناسبة
            for img in train_images:
                shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
            for img in test_images:
                shutil.copy(os.path.join(class_path, img), os.path.join(test_dir, class_name, img))

# تحديد المسارات
base_directory = '/content/drive/MyDrive/Tra'
train_directory = '/content/drive/MyDrive/Training'
test_directory = '/content/drive/MyDrive/Testing'

# تقسيم البيانات
split_data(base_directory, train_directory, test_directory)

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models

# المسار إلى مجلد البيانات
base_directory = '/content/drive/MyDrive/Tra'

# الحصول على عدد الفئات
num_classes = len(os.listdir(base_directory))

# تحميل نموذج VGG16
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# إنشاء نموذج تسلسلي
model = models.Sequential()

# إضافة النموذج الأساسي
model.add(base_model)

# إضافة طبقات جديدة
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))  # طبقة Dropout لتفادي الإفراط في التكيف
model.add(layers.Dense(num_classes, activation='softmax'))  # عدد الفئات

# تجميد الطبقات الأساسية
base_model.trainable = False  # هذا مفيد للبدء في التدريب

# تجميع النموذج
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# ملخص النموذج
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 33)                  │           8,481 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,145,953 (80.67 MB)

 Trainable params: 6,431,265 (24.53 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 33)                  │           8,481 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,145,953 (80.67 MB)

 Trainable params: 6,431,265 (24.53 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [19]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# المسار إلى مجلد البيانات
base_directory = '/content/drive/MyDrive/Tra'  # تأكد من تحديث هذا المسار

# الحصول على عدد الفئات
num_classes = len(os.listdir(base_directory))

# تحميل نموذج VGG16
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# إنشاء نموذج تسلسلي
model = models.Sequential()

# إضافة النموذج الأساسي
model.add(base_model)

# إضافة طبقات جديدة
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))  # طبقة Dropout لتفادي الإفراط في التكيف
model.add(layers.Dense(num_classes, activation='softmax'))  # عدد الفئات

# تجميد الطبقات الأساسية
base_model.trainable = False

# تجميع النموذج
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# إعداد معلمات زيادة البيانات (Data Augmentation)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # تطبيع القيم بين 0 و 1
    validation_split=0.2  # تقسيم البيانات إلى مجموعة تدريب والتحقق
)

# تحميل بيانات التدريب
train_data = train_datagen.flow_from_directory(
    base_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',  # استخدم 'categorical' إذا كانت تسمياتك ذات فئات متعددة
    subset='training'  # استخدام مجموعة التدريب
)

# تحميل بيانات التحقق
val_data = train_datagen.flow_from_directory(
    base_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',  # استخدم 'categorical' إذا كانت تسمياتك ذات فئات متعددة
    subset='validation'  # استخدام مجموعة التحقق
)

# تدريب النموذج
history = model.fit(train_data, epochs=10, validation_data=val_data)

Found 449 images belonging to 33 classes.
Found 97 images belonging to 33 classes.
Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 307s 21s/step - accuracy: 0.0531 - loss: 5.4945 - val_accuracy: 0.1546 - val_loss: 3.1901
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 312s 22s/step - accuracy: 0.1561 - loss: 2.9650 - val_accuracy: 0.2474 - val_loss: 2.6879
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 305s 21s/step - accuracy: 0.3388 - loss: 2.4342 - val_accuracy: 0.4021 - val_loss: 2.3275
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 307s 21s/step - accuracy: 0.3913 - loss: 2.0857 - val_accuracy: 0.4845 - val_loss: 2.1245
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 303s 20s/step - accuracy: 0.5443 - loss: 1.6441 - val_accuracy: 0.4948 - val_loss: 1.9115
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 303s 20s/step - accuracy: 0.5819 - loss: 1.3960 - val_accuracy: 0.5258 - val_loss: 1.8475
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 332s 22s/step - accuracy: 0.6600 - loss: 1.1056 - val_accuracy: 0.5155 - val_loss: 1.6720
Epoch 8/10
15/15 ━━━━

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,  # تدوير الصور حتى 30 درجة
    width_shift_range=0.2,  # إزاحة عرضية
    height_shift_range=0.2,  # إزاحة رأسية
    shear_range=0.2,  # قص الصور
    zoom_range=0.2,  # تكبير الصور
    horizontal_flip=True,  # قلب الصور أفقيًا
    fill_mode='nearest',  # طريقة ملء المساحات الفارغة
    validation_split=0.2
)

In [21]:
# فك تجميد بعض الطبقات
for layer in base_model.layers[-4:]:  # فك تجميد آخر 4 طبقات
    layer.trainable = True

# تغيير معدل التعلم
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-5),  # معدل تعلم منخفض
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# تدريب النموذج مرة أخرى
history_fine = model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 365s 24s/step - accuracy: 0.8407 - loss: 0.5388 - val_accuracy: 0.5876 - val_loss: 1.4956
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 360s 24s/step - accuracy: 0.8694 - loss: 0.4153 - val_accuracy: 0.5979 - val_loss: 1.5119
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 359s 24s/step - accuracy: 0.8697 - loss: 0.3944 - val_accuracy: 0.5876 - val_loss: 1.5160
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 358s 24s/step - accuracy: 0.8829 - loss: 0.4062 - val_accuracy: 0.5979 - val_loss: 1.5096
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 356s 24s/step - accuracy: 0.8221 - loss: 0.4478 - val_accuracy: 0.6289 - val_loss: 1.4990
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 357s 24s/step - accuracy: 0.8868 - loss: 0.3559 - val_accuracy: 0.6392 - val_loss: 1.5121
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 354s 24s/step - accuracy: 0.8683 - loss: 0.3953 - val_accuracy: 0.6186 - val_loss: 1.5314
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 354s 24s/step - accuracy: 0.8556 - loss: 0.4611 - val_accuracy: 0.

In [22]:
# إضافة Batch Normalization
model.add(layers.BatchNormalization())

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_data, epochs=30, validation_data=val_data, callbacks=[early_stopping])

Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 355s 24s/step - accuracy: 0.8745 - loss: 0.3879 - val_accuracy: 0.6186 - val_loss: 1.4909
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 356s 24s/step - accuracy: 0.9367 - loss: 0.2488 - val_accuracy: 0.6082 - val_loss: 1.5183
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 353s 24s/step - accuracy: 0.8719 - loss: 0.3935 - val_accuracy: 0.6186 - val_loss: 1.5404
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 379s 25s/step - accuracy: 0.8944 - loss: 0.2836 - val_accuracy: 0.6186 - val_loss: 1.5393
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 348s 23s/step - accuracy: 0.9096 - loss: 0.2894 - val_accuracy: 0.6082 - val_loss: 1.5276
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 377s 27s/step - accuracy: 0.8979 - loss: 0.2733 - val_accuracy: 0.6082 - val_loss: 1.5521


In [26]:
# حفظ النموذج بالتنسيق الجديد
model.save('my_model.keras')  # استخدم .keras بدلاً من .h5